# Yandex Algorithm 2018 ML track

 https://contest.yandex.ru/algorithm2018/contest/7914/problems/ 

** 6 место private LB (elena-vo@yandex.ru) **

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вспомогательные-функции" data-toc-modified-id="Вспомогательные-функции-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вспомогательные функции</a></span></li><li><span><a href="#Читаем-train_set,-test_set,-соединяем-в-один-датасет" data-toc-modified-id="Читаем-train_set,-test_set,-соединяем-в-один-датасет-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Читаем train_set, test_set, соединяем в один датасет</a></span></li><li><span><a href="#Y" data-toc-modified-id="Y-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Y</a></span></li></ul></li><li><span><a href="#Cleaning-и-новые-статистические-признаки" data-toc-modified-id="Cleaning-и-новые-статистические-признаки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Cleaning и новые статистические признаки</a></span><ul class="toc-item"><li><span><a href="#Уберем-id-по-которым-не-более-одного-хорошего-(l_n&gt;=1)-ответа" data-toc-modified-id="Уберем-id-по-которым-не-более-одного-хорошего-(l_n>=1)-ответа-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Уберем id по которым не более одного хорошего (l_n&gt;=1) ответа</a></span></li><li><span><a href="#Нехорошие-символы" data-toc-modified-id="Нехорошие-символы-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Нехорошие символы</a></span></li><li><span><a href="#Подготовка-текста,-нормализация" data-toc-modified-id="Подготовка-текста,-нормализация-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Подготовка текста, нормализация</a></span></li><li><span><a href="#Сontext_sum,-дубликаты-по-context_sum-с-разными-context_id" data-toc-modified-id="Сontext_sum,-дубликаты-по-context_sum-с-разными-context_id-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Сontext_sum, дубликаты по context_sum с разными context_id</a></span></li><li><span><a href="#Количество-пересекающихся-слов-в-reply-и-контекстах" data-toc-modified-id="Количество-пересекающихся-слов-в-reply-и-контекстах-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Количество пересекающихся слов в reply и контекстах</a></span></li><li><span><a href="#Длины,-гласные,-согласные-и-др.-признаки" data-toc-modified-id="Длины,-гласные,-согласные-и-др.-признаки-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Длины, гласные, согласные и др. признаки</a></span></li></ul></li><li><span><a href="#FastText" data-toc-modified-id="FastText-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>FastText</a></span></li><li><span><a href="#Валидация" data-toc-modified-id="Валидация-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Валидация</a></span><ul class="toc-item"><li><span><a href="#Функции-для-валидации" data-toc-modified-id="Функции-для-валидации-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Функции для валидации</a></span></li><li><span><a href="#Признаки-и-параметры" data-toc-modified-id="Признаки-и-параметры-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Признаки и параметры</a></span></li></ul></li><li><span><a href="#Predict" data-toc-modified-id="Predict-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Predict</a></span></li><li><span><a href="#Submition" data-toc-modified-id="Submition-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Submition</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import re
import pymorphy2

from sklearn.model_selection import KFold, train_test_split
import sklearn

import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

import csv

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Подготовка

In [5]:
PTD = '../data/Ya/'

PTIN = PTD + 'raw/'
PTR = PTD + 'result/'
PTTMP = PTD + '_tmp/'
PTSUB = PTD + 'sub/'

In [6]:
col_service = [
    'context_id',
    'context_2',
    'context_1',
    'context_0',
    'reply_id',
    'reply',
    'label',
    'confidence',
    'is_train',
    'l_n',
    'y', 'context_sum']

In [7]:
n_splits_n, random_state_n = (10, 42)

### Вспомогательные функции

In [8]:
def load_files(pref, ext, suf_out=None, path=PTTMP):
    
    suf = ('' if suf_out==None else "".join((str(suf_out))))
    ext = '.' + ext
    name_file =  "_".join((path, pref, suf,  ext))
    # name_file = PATH_TO_DATA +  'y_valid_'+ suf +'_.pcl'
    #print(name_file)    
    #    X=pd.DataFrame()
    try:
        if ext=='.npz':
            X = scipy.sparse.load_npz(name_file)
        elif ext=='.pcl':
            X = pd.read_pickle(name_file)
        elif ext=='.csv':
            X = pd.read_csv(name_file, index_col=0)
        else:
            print('Error! Расширение файла: ',ext,'мне не понравилось!')
        #print ("ok")
    except Exception as e:
        print('Error!' , str(e))
        # break
    
    return X

In [9]:
def save_df_to_file(df, pref, ext, suf_out=None, path=PTTMP, verbose=True):
    """
    Сохранение на диск файла: имя = _pref_suf_.ext с расширением ext = ['npz','pcl','csv']
    путь = path
    имя = _pref_suf_.ext
    """
    
    suf = ('' if suf_out is None else "".join((str(suf_out))))
    ext = '.' + ext
    
    name_file_out =  "_".join((path, pref, suf,  ext))
    #print(name_file_out) 
    
    try:
        if ext=='.npz':
            scipy.sparse.save_npz(name_file_out, df)
        elif ext=='.pcl':
            df.to_pickle(name_file_out)
        elif ext=='.csv':
            df.to_csv(name_file_out)
        else:
            if verbose: print('Error! Расширение файла',ext,'мне не понравилось!')
        print('Записали на диск файл', name_file_out)
    except Exception as e:
        print('Error!', str(e))
    
    
    return name_file_out

In [10]:
def simple_feature(df, func, cols=['context_2', 'context_1', 'context_0', 'context_sum', 'reply']):
    new_f_names = []
    if (type(func) != list): func = [func]
    
    for col in cols:
        for f in func:
            df[col + '_' + f.__name__] = df[col].apply(f)
            new_f_names.append(col + '_' + f.__name__)
    
    return df, new_f_names

In [11]:
def get_ind_for_model(dt, drop_col=None, col_service=col_service):
    if drop_col is None: drop_col=[]
    return [list(dt.columns).index(item) for item in list(dt.columns)
            if ((item not in col_service) & (item not in drop_col))]

In [12]:
def my_plot_importance(booster, figsize, max_num_features=20, **kwargs):
    from xgboost import plot_importance
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, max_num_features=max_num_features,   ax=ax, **kwargs)

### Читаем train_set, test_set, соединяем в один датасет

In [13]:
tr_cols = ['context_id','context_2','context_1','context_0','reply_id','reply','label','confidence']
te_cols = ['context_id','context_2','context_1','context_0','reply_id','reply']
all_cont_cols = ['context_2','context_1','context_0', 'reply']
new_fs =[]

In [14]:
train = pd.read_csv(PTIN + 'train.tsv', sep = "\t", header = None, names=tr_cols, quoting=csv.QUOTE_NONE)
test = pd.read_csv(PTIN + 'final.tsv', sep = "\t", header = None,names=te_cols, quoting=csv.QUOTE_NONE)

In [15]:
train['is_train'] = np.int32(1)
test['is_train'] = np.int32(0)

In [64]:
train.shape, test.shape, (train.shape[0] + test.shape[0])

((97533, 11), (104834, 7), 202367)

Проверим финальный data-set(test) на наличие 'context_id' из train

In [16]:
tr_cont_ind = list(train.groupby('context_id')['label'].count().index)
train.loc[train.context_id==125857571229545]

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence,is_train
43896,125857571229545,NaN,NaN,на кого я похожа ?,0,на меня .,good,0.687344,1
43897,125857571229545,NaN,NaN,на кого я похожа ?,1,на друга моего брата .,good,0.724717,1


In [17]:
test.loc[test.context_id.isin(tr_cont_ind), :]

,context_id,context_2,context_1,context_0,reply_id,reply,is_train
46862,125857571229545,NaN,NaN,на кого я похожа ?,0,на твою тётю .,0
46863,125857571229545,NaN,NaN,на кого я похожа ?,1,на себя .,0
46864,125857571229545,NaN,NaN,на кого я похожа ?,2,"на милую , нормальную девушку .",0
46865,125857571229545,NaN,NaN,на кого я похожа ?,3,на литл - уэйна .,0


### Y

In [18]:
d_n = {
    'good' : 2, 
    'neutral' : 1, 
    'bad' : 0
}

train['l_n'] = train.label.map(d_n).astype(np.int16)

**'Y' - целое число: 'label' перераспределён на 4 класса c учётом 'confidence.**              
*Вычисление 'y' здесь в два этапа только для наглядности*

In [19]:
EPS = 0.1
train.loc[train['l_n'] > 0, 'y'] = ((train.l_n + 1 + EPS) * train.confidence) / 6.2
train.loc[train['l_n'] == 2, 'y'] = train['y'] * 2 
train.loc[train['l_n'] == 0, 'y'] = (-train.confidence + 1) / 6.2


train.groupby('l_n')['y'].agg([np.min, np.max])

,amin,amax
l_n,,
0,0.000011,0.106646
1,0.117896,0.338686
2,0.344573,0.999899


In [20]:
train['y'] = np.around((train.y * 3.1).astype(np.double)) - 1
train.groupby('l_n')['y'].agg([np.min, np.max])

,amin,amax
l_n,,
0,-1.0,-1.0
1,-1.0,0.0
2,0.0,2.0


In [21]:
dt = pd.concat([train, test])
dt = dt[train.columns]

## Cleaning и новые статистические признаки

In [22]:
dt.fillna('', inplace=True)

###  Уберем id по которым не более одного хорошего (l_n>=1) ответа

In [23]:
cont_train = dt[dt['is_train'] == 1].groupby('context_id')['l_n'].sum()
cont_train = cont_train[cont_train<=1].index

dt = dt.loc[~dt.context_id.isin(cont_train), :]
dt.index = range(len(dt))
dt.shape

(199345, 11)

### Нехорошие символы

In [24]:
vowels = 'аеёиоуыэюя'
consonant = 'бвгджзйклмнпрстфхцчшщъь'
alphabet = vowels + consonant


In [25]:
def symbols(sentence , n):
    res = []
    for s in sentence:
        res.extend(list(s.lower()))
    return set(res)

dt.reply.str.cat([dt.context_1, dt.context_2], sep=' ')
all_symbols = symbols(dt.reply.str.cat([dt.context_1, dt.context_2], sep=' ').values, 1)
bad_symbols = all_symbols ^ set(alphabet) ^ set(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

In [26]:
def bad_chs(x):
    return len([s for s in x.lower() if s in bad_symbols])

dt, new_fs = simple_feature(dt, bad_chs, cols=['context_2', 'context_1', 'context_0', 'reply'])

In [27]:
dt['is_dupl_r'] = dt['reply'].duplicated(keep=False).astype('int8')

### Подготовка текста, нормализация

In [28]:
ws =['что', 'чего', 'чему', 'чем', 'кто','кого', 'кому', 'кем', 'где', 'когда', 'как', 'столько', 'сколько','куда', 
     'первых', 'вторых', 'третьих', 'четрвертых', 'пятых', 'шестых', 'седьмых']
prs = ['кое', 'во']
chs = ['нибудь', 'либо', 'то']

morph = pymorphy2.MorphAnalyzer()

def drop_gap(text):
    for w in ws:
        for pr in prs:
            text = text.replace(pr + ' ' + '-' + ' ' + w, pr + '-' + w)
        for ch in chs:
            text = text.replace(w + ' ' + '-' + ' ' + ch, w + '-' + ch)
    return text


# stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
def review_to_words(review, normalize=True):
    """
    0) заменим <censored> на 'нецензурный'
    01) уберем пробел в 'кто - нибудь' etc
    1)удалим все символы кроме букв верхнего и нижнего регистра,цифр и -;
    2)преобразуем слова к нижнему регистру;
    # 3)удалим стоп слова;
    4)лемматизация
    5)приведем к строке
    
    Результат:
    Возвращает строку из слов (для w2v - последнюю строку убрать)
    """
    
    #0)
    review = review.replace('<CENSORED>', 'нецензурный')
    #01)
    review = drop_gap(review)
    #1)
    # review_ = re.sub("[0-9]","7", review) до sub.18 вкл.
    review_text = re.sub("[^а-яёА-Я0-9\-]"," ", review)
    review_text = review_text.replace(' - ', ' ') # 22.4
    #2)
    words = review_text.lower().split()
    #3)
    # words = [w for w in words if not w in stops]
    #4)
    if normalize: words = [morph.parse(w)[0].normal_form for w in words ]
    words = " ".join(words)
    return words

In [29]:
%%time
cont_rep = ['context_2','context_1','context_0','reply']
for col in cont_rep:
    dt[col] = dt[col].apply(review_to_words)

CPU times: user 1min 14s, sys: 150 ms, total: 1min 14s
Wall time: 1min 14s


### Сontext_sum, дубликаты по context_sum с разными context_id

In [30]:
dt['context_sum'] = dt['context_2'].str.cat([dt['context_1'], dt['context_0']], sep=' ')

In [31]:
dt['is_dupl_sum'] = dt.loc[(dt.reply_id == 0), ['context_sum']].duplicated(keep=False).astype('int8')

dup_ind = dt.loc[dt['is_dupl_sum']==1, :].groupby('context_id')['y'].count().index.tolist()
dt.loc[dt.context_id.isin(dup_ind), 'is_dupl_sum'] = 1
dt.loc[~dt.context_id.isin(dup_ind), 'is_dupl_sum'] = 0

### Количество пересекающихся слов в reply и контекстах

In [32]:
def get_overlap(data_row):
    """
    Возвращает количество n пересекающихся слов в reply и контекстах- dict[context_i] = n 
    data_row - строка, dt.loc[data_ind]
    """
    words = {}
    overlap = {}

    for col in ['context_2', 'context_1', 'context_0', 'reply','context_sum']:
        words[col] = data_row[col].split()
    
    reply_words = set(words['reply'])
        
    for col in ['context_2', 'context_1', 'context_0', 'context_sum']:
        context_words = set(words[col])
        overlap[col] = len(context_words.intersection(reply_words))
                
    return overlap

In [33]:
context_cols = ['context_2', 'context_1', 'context_0', 'context_sum']
overlap_df = pd.DataFrame(columns=context_cols)


for data_ind in range(len(dt)):
    ov = get_overlap(dt.loc[data_ind])
    
    for col in context_cols:
        overlap_df.loc[data_ind, col] = ov[col]

map_ = {col : col + '_ovr' for col in context_cols}
overlap_df = overlap_df.rename(map_, axis=1).astype(np.int16)

In [34]:
dt = pd.concat([dt, overlap_df], axis=1)

In [35]:
def len_ws(x):
    """
    Количество РАЗНЫХ слов 
    """
    return len(set(x.split()))


In [36]:
dt, new_lws_names = simple_feature(dt, len_ws)

In [37]:
def get_ratio(df, col_part, col, suf='_ratio'):
    
    new_col = col_part + '__' + col + suf
    df[new_col] = 0
    df.loc[df[col]!=0, new_col] = df[col_part] / df[col]
    
    return df


In [38]:
cols_part= ['context_2_ovr', 'context_1_ovr', 'context_0_ovr', 'context_sum_ovr']
col = 'reply_len_ws'

for col_part  in cols_part:
    dt = get_ratio(dt, col_part, col)

In [39]:
del overlap_df

### Длины, гласные, согласные и др. признаки

In [40]:
vowels = '[аеёиоуыэюя]'
consonants = '[бвгджзйклмнпрстфхцчшщъь]'


In [41]:
def vow(x):
    return len(re.findall(vowels, x))


def cons(x):
    return len(re.findall(consonants, x))


def div_vov_cons(x):
    if cons(x)!=0:
        return vow(x)/(cons(x))
    else:
        return 0

def len_chs(x):
    return len(''.join(x.split()))


In [42]:
def simple_feature(df, func, cols=['context_2', 'context_1', 'context_0', 'context_sum', 'reply']):
    new_f_names = []
    if (type(func) != list): func = [func]
    
    for col in cols:
        for f in func:
            df[col + '_' + f.__name__] = df[col].apply(f)
            new_f_names.append(col + '_' + f.__name__)
    
    return df, new_f_names

In [43]:
func = [len_chs, vow, cons, div_vov_cons]
dt, new_fsf_names = simple_feature(dt, func)

In [44]:
dt['context_sum_bad_chs'] = dt[['context_2_bad_chs', 'context_1_bad_chs', 'context_0_bad_chs']].sum(axis=1)

In [45]:
func = [len_chs, vow, cons, bad_chs]
cont_cols=['context_2', 'context_1', 'context_0', 'context_sum']

for f in func:
    col_part = 'reply_' + f.__name__
    cols = [c + '_' + f.__name__ for c in cont_cols]
    
    for col  in cols:
        try:
            dt = get_ratio(dt, col_part, col)
        except:
            print(col)

In [46]:
col_for_dif = {}
col_r = {}

col_for_dif[0] = ['context_2_len_ws', 'context_1_len_ws', 'context_0_len_ws', 'context_sum_len_ws']
col_r[0] = 'reply_len_ws'

col_for_dif[1] = ['context_2_len_chs', 'context_1_len_chs', 'context_0_len_chs', 'context_sum_len_chs']
col_r[1] = 'reply_len_chs'

col_for_dif[2] = ['context_0_vow', 'context_sum_vow']
col_r[2] = 'reply_vow'

col_for_dif[3] = ['context_0_cons', 'context_sum_cons']
col_r[3] = 'reply_cons'

In [47]:
def get_dif(df, col_c, col_r):
    
    new_col = col_c + '-' + col_r
    df[new_col] = df[col_c] - df[col_r]
    
    return df

def get_part(df, cols):
    for col in cols:
        df[col] = dt[col].map(df[col].value_counts(normalize=True))
    return df

def get_log(df, cols):
    for col in cols:
        df[col] = np.log(dt[col] + 1)
    return df
    


In [48]:
for i in range(4):
    for c in col_for_dif[i]:
        dt = get_dif(dt, c, col_r[i])

In [49]:
cols_for_part = ['is_dupl_r', 
                 'context_2_bad_chs', 'context_1_bad_chs', 'context_0_bad_chs', 'reply_bad_chs', 
                 'context_0_ovr', 'context_1_ovr', 'context_2_ovr', 'context_sum_ovr',
                 ]

In [50]:
cols_for_log = ['context_2_len_ws', 'context_2_len_chs', 'context_2_vow','context_2_cons',
                'context_1_len_ws', 'context_1_len_chs', 'context_1_vow','context_1_cons',
                'context_0_len_ws', 'context_0_len_chs', 'context_0_vow','context_0_cons',
                'context_sum_len_ws', 'context_sum_len_chs', 'context_sum_vow','context_sum_cons',
                'reply_len_ws', 'reply_len_chs', 'reply_vow', 'reply_cons']

In [51]:
dt = get_part(dt, cols_for_part)
dt = get_log(dt, cols_for_log)

## FastText

In [52]:
import fastText as ft
from fastText import load_model

In [53]:
model_ff = load_model('/home/maatkara/fastText/cc_vec/cc.ru.300.bin')

In [54]:
def get_vect(x):
    return model_ff.get_sentence_vector(x)


In [55]:
cols=['context_id', 'context_2', 'context_1', 'context_0', 'context_sum', 'reply_id', 'reply']
all_context_col = ['context_2', 'context_1', 'context_0', 'context_sum','reply']
context_col = ['context_2', 'context_1', 'context_0', 'context_sum']

In [56]:
new_col = []
ft_d = {}
for col in all_context_col:
    ft_d[col] = np.vstack(dt[col].apply(get_vect))

In [57]:
cs_cols = ['c_' + str(i) for i in range(300)]
fts = pd.DataFrame(ft_d['context_sum'],columns=cs_cols)

r_cols = ['r_' + str(i) for i in range(300)]
ftr = pd.DataFrame(ft_d['reply'],columns=r_cols)

In [58]:
dt = pd.concat([dt, fts, ftr], axis=1)
dt.shape

(199345, 680)

In [59]:
del model_ff

## Валидация

### Функции для валидации

In [15]:
def dcg(label):
    
    dcg = 0
    for i in range(len(label)):
        if label[i] > 0:
            dcg += float(label[i] / np.log2(i+2))
    return dcg


def ndcg_p(x):
    
    lab = x.values.tolist()
    best_lab = x.values.tolist()
    best_lab.sort()
    best_lab.reverse()
    
    label, best_label = dcg(lab), dcg(best_lab)
    if label != 0 and best_label != 0:
        return label/best_label
    else:
        return 0
    
    
def score_(vl):

    vl = vl.sort_values(by=['context_id', 'y_pred'], ascending=False)
    scr = vl.groupby('context_id')['l_n'].agg(ndcg_p).mean()
    
    return scr * 100000


In [16]:
def get_matrix_cv_numerical(data_name, n_splits=n_splits_n, random_state=random_state_n,
                            drop_col=None, col_service=col_service, with_ft=False):
    """
    Создадим матрицы для валидации и запишем на диск
    Возвращает res_tr, res_vl  - списки с именами файлов
    """
    i = 0 
    res_tr = []
    res_vl = []
    
   
    
    data = load_files(data_name, 'csv')
    
    ind = get_ind_for_model(dt, drop_col=drop_col, col_service=col_service)
    xs = list(data.columns[ind])
    
    if with_ft: 
        x_name_suf = 'with_ft'
    else:
        xs = xs[:-600]
        x_name_suf = 'without_ft'
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    cont_train = data[data['is_train'] == 1].groupby('context_id')['y'].sum().index
    
    
    #разобъем train на train/valid для валидации по 'context_id'
    for train_index, test_index in kf.split(cont_train):
    
        tr = pd.DataFrame(cont_train[train_index], columns = ['context_id']).merge(data.loc[data.is_train==1,:],
                                                                                   how = 'left')
        vl = pd.DataFrame(cont_train[test_index], columns = ['context_id']).merge(data.loc[data.is_train==1,:],
                                                                                  how = 'left')
        
        
        X_tr_cv = tr[xs]
        X_vl_cv = vl[xs]
        
        res_tr.append(save_df_to_file(X_tr_cv, 'X_tr_cv_' + x_name_suf , 'csv', suf_out=i, verbose=False))
        res_vl.append(save_df_to_file(X_vl_cv, 'X_vl_cv_' + x_name_suf, 'csv', suf_out=i, verbose=False))
        print(X_tr_cv.shape, X_vl_cv.shape)       
        i += 1
    
    return res_tr, res_vl

In [17]:
def validate_num(data,  param, num_round, n_splits=n_splits_n, drop_col=None, random_state=random_state_n, with_ft=True):
    
    
    if with_ft: 
        x_name_suf = 'with_ft'
    else:
        x_name_suf = 'without_ft'

    
    i = 0 
    score = 0
    res = []
    score_l = []
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    y = data.loc[data.is_train == 1, 'y'].values
    
    ind = get_ind_for_model(data, drop_col=drop_col, col_service=col_service)
    xs = list(data.columns[ind])
    
    
    cont_train = data[data['is_train'] == 1].groupby('context_id')['l_n'].count().index
    
    #разобъем train на train/valid для валидации по 'context_id'
    for train_index, test_index in kf.split(cont_train):
        
        if i!=25: # для возможности "отключения" одной из папок
            tr = pd.DataFrame(cont_train[train_index], columns = ['context_id']).merge(data.loc[data.is_train==1,:],
                                                                                       how = 'left')
            vl = pd.DataFrame(cont_train[test_index], columns = ['context_id']).merge(data.loc[data.is_train==1,:],
                                                                                      how = 'left')
        
            gr_tr = tr.groupby('context_id')['l_n'].count().values
            gr_vl = vl.groupby('context_id')['l_n'].count().values
        
            X_tr_cv = load_files('X_tr_cv_' + x_name_suf, 'csv', suf_out=i)
            X_vl_cv = load_files('X_vl_cv_' + x_name_suf, 'csv', suf_out=i)
        
            y_tr = tr['y']
            y_vl = vl['y']
            
            dtrain = xgb.DMatrix(X_tr_cv[xs], label=y_tr)
            dvalid = xgb.DMatrix(X_vl_cv[xs], label=y_vl)
            
            dtrain.set_group(gr_tr)
            dvalid.set_group(gr_vl)
        
            evallist = [(dvalid, 'eval'), (dtrain, 'train')]
        
            clf = xgb.train(param, dtrain, num_round, evallist,  verbose_eval=0)
            vl['y_pred'] = clf.predict(dvalid)
        
            score = score_(vl)
            score_l.append(score)
        i +=1
    
    res = {'mean' : np.mean(score_l), 
           'std' : np.std(score_l), 
           'score_list' : score_l}
    print('-'*50)
    print(res['mean'], ' mean, std=', res['std'], ' max=', np.max(score_l), ' min=', np.min(score_l))
    print('-'*50)
    return res
   

In [18]:
res_tr_n, res_vl_n = get_matrix_cv_numerical('dt', n_splits=n_splits_n, with_ft=True)

### Признаки и параметры 

Исключаем признаки

In [68]:
drop_col = ['context_sum_bad_chs',
 'reply_len_chs__context_2_len_chs_ratio',
 'reply_len_chs__context_1_len_chs_ratio',
 'reply_len_chs__context_0_len_chs_ratio',
 'reply_len_chs__context_sum_len_chs_ratio',
 'reply_vow__context_2_vow_ratio',
 'reply_vow__context_1_vow_ratio',
 'reply_vow__context_0_vow_ratio',
 'reply_vow__context_sum_vow_ratio',
 'reply_cons__context_2_cons_ratio',
 'reply_cons__context_1_cons_ratio',
 'reply_cons__context_0_cons_ratio',
 'reply_cons__context_sum_cons_ratio',
 'reply_bad_chs__context_2_bad_chs_ratio',
 'reply_bad_chs__context_1_bad_chs_ratio',
 'reply_bad_chs__context_0_bad_chs_ratio',
 'reply_bad_chs__context_sum_bad_chs_ratio']

In [28]:
param = {
    'max_depth': 8,
    'eta': 0.05,
    'colsample_bytree': 1,
    'min_child_weight': 7, 
    'subsample': 1, 
    'gamma': 1, 
    
    'booster': 'gbtree',
    'silent':1, 
    'objective':'rank:pairwise', 
    'eval_metric': 'ndcg',
    'nthread':8,
    'seed' : 42,
}

num_round = 100 

In [33]:
validate_num(dt, param, num_round, drop_col=drop_col, with_ft=True)

## Predict

In [65]:
param = {
    'max_depth': 9, 
    'eta': 0.005 , 
    'colsample_bytree': 1,
    'min_child_weight': 7, 
    'subsample': 1, 
    'gamma': 1, 
    
    'booster': 'gbtree',
    'silent':1, 
    'objective':'rank:pairwise', 
    'eval_metric': 'ndcg',
    'nthread':8,
    'seed' : 0,
}
with_ft = True

In [66]:
num_round = 4000 

In [69]:
ind = get_ind_for_model(dt, drop_col=drop_col, col_service=col_service)
xs = list(dt.columns[ind])
    
if with_ft: 
    x_name_suf = 'with_ft'
else:
    xs = xs[:-600]
    x_name_suf = 'without_ft'

    
X_tr = dt.loc[dt.is_train == 1, xs]
X_te = dt.loc[dt.is_train == 0, xs]

len(xs), X_tr.shape, X_te.shape

(651, (94511, 651), (104834, 651))

In [70]:
gr_train = dt.loc[dt.is_train == 1, :].groupby('context_id')['y'].count().values
gr_test  = dt.loc[dt.is_train == 0, :].groupby('context_id')['y'].count().values

In [71]:
y_tr = dt.loc[dt.is_train == 1, 'y']

dtrain = xgb.DMatrix(X_tr, label=y_tr) 
dvalid = xgb.DMatrix(X_te) 
dtrain.set_group(gr_train)
dvalid.set_group(gr_test)
        
clf1 = xgb.train(param, dtrain, num_round)
te_pr = dt.loc[dt.is_train == 0, ['context_id', 'reply_id']]
te_pr['y_pred'] = - clf1.predict(dvalid)

## Submition

In [55]:
te_pr = te_pr.sort_values(by=['context_id', 'y_pred'])
te_pr.head()

,context_id,reply_id,y_pred
94513,4909294510,2,-1.248553
94512,4909294510,1,-1.246581
94515,4909294510,4,-0.965466
94514,4909294510,3,-0.442126
94511,4909294510,0,-0.338845


In [56]:
sub = te_pr[['context_id', 'reply_id']]

In [58]:
sub.shape, dt.loc[dt.is_train == 0, 'y'].shape[0]

((104834, 2), 104834)

In [59]:
def write_submission(preds, number_sub, index=False, header=True):
    
    preds = pd.DataFrame(preds)
    output = PTSUB + 'submition_' + str(number_sub) + '.csv'
    preds.to_csv(output, sep=' ', index=index, header=header)
    clf1.save_model(PTR + 'xgb' + '_'+  str(number_sub) + '.model')
    
    print('Sub ', number_sub, ' in ', output)

In [60]:
number_sub = 53

In [30]:
write_submission(sub, number_sub, header=None)